![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
from AlgorithmImports import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import plotly.graph_objects as go
from datetime import datetime

# 此代码测试了GC连续合约的计算逻辑

# ------------------------------------------------------
# 0. 通用设置
# ------------------------------------------------------
qb = QuantBook()
qb.set_time_zone(TimeZones.HONG_KONG)

start = datetime(2024, 11,27)
end = datetime(2024, 12,1)

# 统一的历史数据查询参数
history_args = dict(
    start=start,
    end=end,
    resolution=Resolution.SECOND,
    extended_market_hours=True,
    data_mapping_mode=DataMappingMode.OPEN_INTEREST_ANNUAL, # 数据映射模式，这个会根据交易量切换到当年后续更大的合约，Warning, 但正确性有待验证
    data_normalization_mode=DataNormalizationMode.BACKWARDS_RATIO, # 数据连续模式，ATAS是RAW, tradingview 是BACKWARDS_RATIO，能够使得连续。注意，实盘需要使用当期合约数据
    fill_forward=True
)


# ------------------------------------------------------
# 1. C0 (近月合约) 数据获取与绘图
# ------------------------------------------------------
print("--- 1. C0 (近月) 数据 ---")
# C0: contract_depth_offset=0
future_c0 = qb.add_future(Futures.Metals.Gold, Resolution.SECOND, contract_depth_offset=0)
continuous_symbol_c0 = future_c0.Symbol  # C0 连续合约 Symbol

history = qb.history(SymbolChangedEvent, continuous_symbol_c0, start,end)
print(history)


df_c0 = qb.history(continuous_symbol_c0, contract_depth_offset=0, **history_args).reset_index()

print(f"C0 原始数据行数: {len(df_c0)}")
# print(df_c0['open'].head()) # 打印部分数据供检查

# 创建 Candlestick 对象 (C0)
candlestick_c0 = go.Candlestick(
    x=df_c0['time'], 
    open=df_c0['open'], 
    high=df_c0['high'], 
    low=df_c0['low'], 
    close=df_c0['close'],
    name='C0 Candlestick'
)

# 定义图表布局 (C0)
layout_c0 = go.Layout(
    title=f'C0 (Front Month) Candlestick Chart: {continuous_symbol_c0.Value}',
    xaxis=dict(title='Date (HKT)'),
    yaxis=dict(title='Price (USD)'),
    xaxis_rangeslider_visible=False
)

# 创建并显示图表 (C0)
fig_c0 = go.Figure(data=[candlestick_c0], layout=layout_c0)
fig_c0.show()
